<a href="https://colab.research.google.com/github/KshKnsl/MinorProject/blob/main/ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install aiohttp pyngrok

import os
import asyncio
import aiohttp

os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken','32eB7tLSQoICKJD4JSQuJ9lWea6_7U5ndjtQCVaWnPLEc4Mws'])
)

ngrok_task_for_url = asyncio.create_task(run_process(['ngrok', 'http', '--log', 'stderr', '--url=mari-unbequeathed-milkily.ngrok-free.app', '11434', '--host-header', 'localhost:11434']))

await asyncio.sleep(5)

try:
    async with aiohttp.ClientSession() as session:
        async with session.get('http://localhost:4040/api/tunnels') as resp:
            data = await resp.json()
            ngrok_url = None
            for tunnel in data['tunnels']:
                if tunnel['proto'] == 'https':
                    ngrok_url = tunnel['public_url']
                    break
            if ngrok_url:
                print(f"\nNgrok Public URL: {ngrok_url}\n")
                os.environ['OLLAMA_ORIGINS'] = ngrok_url
            else:
                print("\nCould not find an HTTPS ngrok tunnel.\n")

except Exception as e:
    print(f"\nCould not retrieve ngrok public URL: {e}\n")
    ngrok_url = None

ollama_task = asyncio.create_task(run_process(['ollama', 'serve']))

await asyncio.gather(ollama_task, ngrok_task_for_url)

>>> starting ngrok config add-authtoken 32eB7tLSQoICKJD4JSQuJ9lWea6_7U5ndjtQCVaWnPLEc4Mws
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ngrok http --log stderr --url=mari-unbequeathed-milkily.ngrok-free.app 11434 --host-header localhost:11434
t=2025-09-15T12:13:17+0000 lvl=info msg="no configuration paths supplied"
t=2025-09-15T12:13:17+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2025-09-15T12:13:17+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2025-09-15T12:13:17+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2025-09-15T12:13:17+0000 lvl=info msg="client session established" obj=tunnels.session
t=2025-09-15T12:13:17+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2025-09-15T12:13:18+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://mari-unbequeathed-